In [4]:
import sys
sys.path.append('../algorithm')

In [5]:
# Load in CIFAR dataset
from datasets import load_data
from config import Dataset
train_data, val_data, test_data = load_data("../data",
                                            Dataset.CIFAR,
                                            subset=0.8,  # irrelevant for this task
                                            seed=0
                                           )

seed: 0
shuffle: [1670, 13379, 10234, 4719, 7003, 2831, 13014, 11979, 8610, 519, ...]


In [6]:
# Apply known noise to the labels
# Let's first try the same noise as the MNIST_FASHION_05 has
import numpy as np

test_labels = test_data.tensors[1]
test_labels_clean = test_labels.clone().detach()
T = np.array([[0.5, 0.3, 0.2], [0.2, 0.5, 0.3], [0.3, 0.2, 0.5]])
for i in range(len(test_labels)):
    # Given this is the true label, use the corresponding noisy probs to get a noisy label
    test_labels[i] = np.random.choice(np.array([0, 1, 2]), p=T[test_labels[i]])

In [7]:
# Check how many are equal to their original value (should be roughly 0.5)
(test_labels_clean == test_labels).sum() / len(test_labels)

tensor(0.4953)

In [8]:
# Here we want to use the test data as our `training data` for estimating the transition matrix
train_data = test_data

In [35]:
# See how our transition matrix esitmator performs on this data
import argparse
import hashlib
import json
import os.path
import sys
from copy import copy
from datetime import datetime
from pathlib import Path
from typing import Callable, Iterable, List, Optional

import numpy as np
import torch
import torch.nn.functional as F
import wandb
from datasets import load_data
from factories import BackboneFactory, EstimatorFactory, LossFactory
from loggers import JSONLLogger, Logger, StreamLogger, WandbLogger
from sklearn.metrics import accuracy_score
from termcolor import colored
from torch.utils.data import DataLoader
from utils import LabelSmoothingCrossEntropyLoss

from config import Backbone, Dataset, Estimator, LossCorrection

anchor_trans_thresh = 0.9
# Adjust params here
config_dict = {'dataset': Dataset.CIFAR, 'subset': 0.8, 'seed': 0, 'loss_correction': LossCorrection.FORWARD, 'backbone': Backbone.MLP, 'estimator': Estimator.ANCHOR, 'freeze_estimator': True, 'anchor_outlier_threshold': anchor_trans_thresh, 'epochs': 0, 'lr': 0.01, 'backbone_pretrain_epochs': 150, 'label_smoothing': 0.0, 'batch_size': 32, 'device': 'cpu', 'id': '20201118_164657', 'log_step': 32, 'results_dir': '/Users/nick/uni/comp5328/assignment2/comp-5328-assignment-two/code/results', 'wandb': False}
config = argparse.Namespace()
vars(config).update(config_dict)


def generate_trans_cifar(config):
    loggers = [StreamLogger()]
    input_size = tuple(train_data.tensors[0].size()[1:])
    class_count = len(set(train_data.tensors[1].tolist()))
    criterion = torch.nn.CrossEntropyLoss()
    
    # Create backbone
    print(colored("backbone:", attrs=["bold"]))
    backbone_factory = BackboneFactory(input_size, class_count)
    backbone = backbone_factory.create(config)
    print(backbone)
    
    # Perform pretraining on noisy data without the transition matrix if necessary
    if config.backbone_pretrain_epochs > 0:
        print(colored("pretraining backbone:", attrs=["bold"]))
        backbone = backbone.to(config.device)
        pretrain_backbone(
            backbone,
            train_data,
            torch.optim.SGD(backbone.parameters(), lr=1e-3),
            criterion,
            loggers,
            config,
        )

    # Estimator could be None if we don't want to use a transition matrix
    # Create transition matrix
    print(colored("estimator:", attrs=["bold"]))
    estimator_factory = EstimatorFactory(class_count)
    estimator = estimator_factory.create(
        config,
        backbone,
        DataLoader(train_data, batch_size=config.batch_size, shuffle=False, num_workers=0),
    )
    if estimator is not None:
        print(
            f"Transition matrix to be usd by the Label Noise Robust Model:\n"
            f"{estimator.transitions=}"
        )






def pretrain_backbone(
    backbone: torch.nn.Module,
    data: torch.utils.data.Dataset,
    optimiser: torch.optim.Optimizer,
    criterion: Optional[Callable[..., torch.Tensor]],
    loggers: Iterable[Logger],
    config: argparse.Namespace,
):
    """Pretrain a backbone model."""
    dataloader = DataLoader(data, batch_size=config.batch_size, shuffle=True, num_workers=0)
    class_count = len(set(dataloader.dataset.tensors[1].tolist()))
    backbone.train()
    for epoch in range(config.backbone_pretrain_epochs):
        for batch, (feats, labels) in enumerate(dataloader):
            # Move data to GPU
            feats = feats.to(config.device)
            # Convert labels to one-hots if using BCEWithLogitsLoss
            if isinstance(criterion, torch.nn.BCEWithLogitsLoss):
                labels = F.one_hot(labels, num_classes=class_count).type(torch.float32)
            labels = labels.to(config.device)
            optimiser.zero_grad()
            noisy_posteriors, noisy_activations = backbone(feats)
            loss = criterion(noisy_activations, labels)
            loss.backward()

            optimiser.step()

            if batch % config.log_step == config.log_step - 1 or batch == len(dataloader) - 1:
                metrics = {
                    "pretrain/epoch": epoch
                    + batch * dataloader.batch_size / len(dataloader.dataset),
                    "pretrain/loss": loss.item(),
                }
                for logger in loggers:
                    logger(metrics)
                    
                    
generate_trans_cifar(config)

backbone:
MLPBackbone(
  (fc_0): Linear(in_features=3072, out_features=1537, bias=True)
  (act_0): ReLU()
  (dropout_0): Dropout(p=0.5, inplace=False)
  (fc_1): Linear(in_features=1537, out_features=1537, bias=True)
  (act_1): ReLU()
  (fc_2): Linear(in_features=1537, out_features=3, bias=True)
  (sm): Softmax(dim=-1)
)
pretraining backbone:
pretrain/epoch: 0.3307 pretrain/loss: 1.0928
pretrain/epoch: 0.6720 pretrain/loss: 1.1056
pretrain/epoch: 0.9920 pretrain/loss: 1.0942
pretrain/epoch: 1.3307 pretrain/loss: 1.1005
pretrain/epoch: 1.6720 pretrain/loss: 1.1003
pretrain/epoch: 1.9920 pretrain/loss: 1.1045
pretrain/epoch: 2.3307 pretrain/loss: 1.0971
pretrain/epoch: 2.6720 pretrain/loss: 1.1055
pretrain/epoch: 2.9920 pretrain/loss: 1.1051
pretrain/epoch: 3.3307 pretrain/loss: 1.1137
pretrain/epoch: 3.6720 pretrain/loss: 1.0932
pretrain/epoch: 3.9920 pretrain/loss: 1.1071
pretrain/epoch: 4.3307 pretrain/loss: 1.0997
pretrain/epoch: 4.6720 pretrain/loss: 1.0976
pretrain/epoch: 4.9920 pre

pretrain/epoch: 41.3307 pretrain/loss: 1.0568
pretrain/epoch: 41.6720 pretrain/loss: 1.0863
pretrain/epoch: 41.9920 pretrain/loss: 1.1105
pretrain/epoch: 42.3307 pretrain/loss: 1.0621
pretrain/epoch: 42.6720 pretrain/loss: 1.0806
pretrain/epoch: 42.9920 pretrain/loss: 1.0669
pretrain/epoch: 43.3307 pretrain/loss: 1.0524
pretrain/epoch: 43.6720 pretrain/loss: 1.0838
pretrain/epoch: 43.9920 pretrain/loss: 1.0640
pretrain/epoch: 44.3307 pretrain/loss: 1.1182
pretrain/epoch: 44.6720 pretrain/loss: 1.1366
pretrain/epoch: 44.9920 pretrain/loss: 1.0670
pretrain/epoch: 45.3307 pretrain/loss: 1.0731
pretrain/epoch: 45.6720 pretrain/loss: 1.0688
pretrain/epoch: 45.9920 pretrain/loss: 1.0580
pretrain/epoch: 46.3307 pretrain/loss: 1.0771
pretrain/epoch: 46.6720 pretrain/loss: 1.0626
pretrain/epoch: 46.9920 pretrain/loss: 1.0733
pretrain/epoch: 47.3307 pretrain/loss: 1.1018
pretrain/epoch: 47.6720 pretrain/loss: 1.0956
pretrain/epoch: 47.9920 pretrain/loss: 1.0761
pretrain/epoch: 48.3307 pretrain/l

pretrain/epoch: 84.3307 pretrain/loss: 1.0549
pretrain/epoch: 84.6720 pretrain/loss: 1.0823
pretrain/epoch: 84.9920 pretrain/loss: 1.0713
pretrain/epoch: 85.3307 pretrain/loss: 1.1434
pretrain/epoch: 85.6720 pretrain/loss: 1.0894
pretrain/epoch: 85.9920 pretrain/loss: 1.0734
pretrain/epoch: 86.3307 pretrain/loss: 1.0807
pretrain/epoch: 86.6720 pretrain/loss: 1.1141
pretrain/epoch: 86.9920 pretrain/loss: 1.1122
pretrain/epoch: 87.3307 pretrain/loss: 1.0506
pretrain/epoch: 87.6720 pretrain/loss: 1.0418
pretrain/epoch: 87.9920 pretrain/loss: 1.0404
pretrain/epoch: 88.3307 pretrain/loss: 1.1091
pretrain/epoch: 88.6720 pretrain/loss: 1.0505
pretrain/epoch: 88.9920 pretrain/loss: 1.0789
pretrain/epoch: 89.3307 pretrain/loss: 1.0359
pretrain/epoch: 89.6720 pretrain/loss: 1.0965
pretrain/epoch: 89.9920 pretrain/loss: 1.0363
pretrain/epoch: 90.3307 pretrain/loss: 1.0829
pretrain/epoch: 90.6720 pretrain/loss: 1.0207
pretrain/epoch: 90.9920 pretrain/loss: 1.0943
pretrain/epoch: 91.3307 pretrain/l

pretrain/epoch: 126.6720 pretrain/loss: 1.0024
pretrain/epoch: 126.9920 pretrain/loss: 1.0539
pretrain/epoch: 127.3307 pretrain/loss: 1.0610
pretrain/epoch: 127.6720 pretrain/loss: 1.1175
pretrain/epoch: 127.9920 pretrain/loss: 1.0507
pretrain/epoch: 128.3307 pretrain/loss: 1.0141
pretrain/epoch: 128.6720 pretrain/loss: 1.0853
pretrain/epoch: 128.9920 pretrain/loss: 1.0504
pretrain/epoch: 129.3307 pretrain/loss: 1.0373
pretrain/epoch: 129.6720 pretrain/loss: 1.0152
pretrain/epoch: 129.9920 pretrain/loss: 1.0733
pretrain/epoch: 130.3307 pretrain/loss: 1.0399
pretrain/epoch: 130.6720 pretrain/loss: 1.0370
pretrain/epoch: 130.9920 pretrain/loss: 1.0121
pretrain/epoch: 131.3307 pretrain/loss: 0.9928
pretrain/epoch: 131.6720 pretrain/loss: 1.0449
pretrain/epoch: 131.9920 pretrain/loss: 1.0394
pretrain/epoch: 132.3307 pretrain/loss: 1.0654
pretrain/epoch: 132.6720 pretrain/loss: 1.0190
pretrain/epoch: 132.9920 pretrain/loss: 0.9629
pretrain/epoch: 133.3307 pretrain/loss: 1.0687
pretrain/epoc

In [36]:
anchor_trans_thresh = 0.97
# Adjust params here
config_dict = {'dataset': Dataset.CIFAR, 'subset': 0.8, 'seed': 0, 'loss_correction': LossCorrection.FORWARD, 'backbone': Backbone.MLP, 'estimator': Estimator.ANCHOR, 'freeze_estimator': True, 'anchor_outlier_threshold': anchor_trans_thresh, 'epochs': 0, 'lr': 0.01, 'backbone_pretrain_epochs': 150, 'label_smoothing': 0.0, 'batch_size': 32, 'device': 'cpu', 'id': '20201118_164657', 'log_step': 32, 'results_dir': '/Users/nick/uni/comp5328/assignment2/comp-5328-assignment-two/code/results', 'wandb': False}
config = argparse.Namespace()
vars(config).update(config_dict)

generate_trans_cifar(config)



backbone:
MLPBackbone(
  (fc_0): Linear(in_features=3072, out_features=1537, bias=True)
  (act_0): ReLU()
  (dropout_0): Dropout(p=0.5, inplace=False)
  (fc_1): Linear(in_features=1537, out_features=1537, bias=True)
  (act_1): ReLU()
  (fc_2): Linear(in_features=1537, out_features=3, bias=True)
  (sm): Softmax(dim=-1)
)
pretraining backbone:
pretrain/epoch: 0.3307 pretrain/loss: 1.1194
pretrain/epoch: 0.6720 pretrain/loss: 1.0971
pretrain/epoch: 0.9920 pretrain/loss: 1.0907
pretrain/epoch: 1.3307 pretrain/loss: 1.0950
pretrain/epoch: 1.6720 pretrain/loss: 1.1030
pretrain/epoch: 1.9920 pretrain/loss: 1.0971
pretrain/epoch: 2.3307 pretrain/loss: 1.0922
pretrain/epoch: 2.6720 pretrain/loss: 1.1094
pretrain/epoch: 2.9920 pretrain/loss: 1.0973
pretrain/epoch: 3.3307 pretrain/loss: 1.0894
pretrain/epoch: 3.6720 pretrain/loss: 1.1137
pretrain/epoch: 3.9920 pretrain/loss: 1.1058
pretrain/epoch: 4.3307 pretrain/loss: 1.0972
pretrain/epoch: 4.6720 pretrain/loss: 1.1058
pretrain/epoch: 4.9920 pre

pretrain/epoch: 41.3307 pretrain/loss: 1.0459
pretrain/epoch: 41.6720 pretrain/loss: 1.0910
pretrain/epoch: 41.9920 pretrain/loss: 1.1178
pretrain/epoch: 42.3307 pretrain/loss: 1.0832
pretrain/epoch: 42.6720 pretrain/loss: 1.0640
pretrain/epoch: 42.9920 pretrain/loss: 1.0747
pretrain/epoch: 43.3307 pretrain/loss: 1.0912
pretrain/epoch: 43.6720 pretrain/loss: 1.0436
pretrain/epoch: 43.9920 pretrain/loss: 1.0736
pretrain/epoch: 44.3307 pretrain/loss: 1.0543
pretrain/epoch: 44.6720 pretrain/loss: 1.0741
pretrain/epoch: 44.9920 pretrain/loss: 1.0458
pretrain/epoch: 45.3307 pretrain/loss: 1.1114
pretrain/epoch: 45.6720 pretrain/loss: 1.0777
pretrain/epoch: 45.9920 pretrain/loss: 1.0568
pretrain/epoch: 46.3307 pretrain/loss: 1.0884
pretrain/epoch: 46.6720 pretrain/loss: 1.0965
pretrain/epoch: 46.9920 pretrain/loss: 1.0935
pretrain/epoch: 47.3307 pretrain/loss: 1.0694
pretrain/epoch: 47.6720 pretrain/loss: 1.1101
pretrain/epoch: 47.9920 pretrain/loss: 1.0545
pretrain/epoch: 48.3307 pretrain/l

pretrain/epoch: 84.3307 pretrain/loss: 1.0872
pretrain/epoch: 84.6720 pretrain/loss: 1.0730
pretrain/epoch: 84.9920 pretrain/loss: 1.0073
pretrain/epoch: 85.3307 pretrain/loss: 1.0496
pretrain/epoch: 85.6720 pretrain/loss: 1.1048
pretrain/epoch: 85.9920 pretrain/loss: 1.0774
pretrain/epoch: 86.3307 pretrain/loss: 1.0716
pretrain/epoch: 86.6720 pretrain/loss: 1.0396
pretrain/epoch: 86.9920 pretrain/loss: 1.0247
pretrain/epoch: 87.3307 pretrain/loss: 1.0458
pretrain/epoch: 87.6720 pretrain/loss: 1.0380
pretrain/epoch: 87.9920 pretrain/loss: 1.0795
pretrain/epoch: 88.3307 pretrain/loss: 1.1142
pretrain/epoch: 88.6720 pretrain/loss: 1.1126
pretrain/epoch: 88.9920 pretrain/loss: 1.0674
pretrain/epoch: 89.3307 pretrain/loss: 1.0755
pretrain/epoch: 89.6720 pretrain/loss: 1.0477
pretrain/epoch: 89.9920 pretrain/loss: 1.0657
pretrain/epoch: 90.3307 pretrain/loss: 1.0574
pretrain/epoch: 90.6720 pretrain/loss: 1.0631
pretrain/epoch: 90.9920 pretrain/loss: 1.0817
pretrain/epoch: 91.3307 pretrain/l

pretrain/epoch: 126.6720 pretrain/loss: 1.0850
pretrain/epoch: 126.9920 pretrain/loss: 1.0894
pretrain/epoch: 127.3307 pretrain/loss: 1.0747
pretrain/epoch: 127.6720 pretrain/loss: 1.0234
pretrain/epoch: 127.9920 pretrain/loss: 1.0111
pretrain/epoch: 128.3307 pretrain/loss: 0.9773
pretrain/epoch: 128.6720 pretrain/loss: 1.0554
pretrain/epoch: 128.9920 pretrain/loss: 1.1408
pretrain/epoch: 129.3307 pretrain/loss: 1.0608
pretrain/epoch: 129.6720 pretrain/loss: 1.1488
pretrain/epoch: 129.9920 pretrain/loss: 0.9669
pretrain/epoch: 130.3307 pretrain/loss: 1.0607
pretrain/epoch: 130.6720 pretrain/loss: 1.1439
pretrain/epoch: 130.9920 pretrain/loss: 1.0076
pretrain/epoch: 131.3307 pretrain/loss: 1.0546
pretrain/epoch: 131.6720 pretrain/loss: 1.0672
pretrain/epoch: 131.9920 pretrain/loss: 1.1258
pretrain/epoch: 132.3307 pretrain/loss: 1.1008
pretrain/epoch: 132.6720 pretrain/loss: 0.9978
pretrain/epoch: 132.9920 pretrain/loss: 1.0892
pretrain/epoch: 133.3307 pretrain/loss: 1.0590
pretrain/epoc

In [37]:
anchor_trans_thresh = 1
# Adjust params here
config_dict = {'dataset': Dataset.CIFAR, 'subset': 0.8, 'seed': 0, 'loss_correction': LossCorrection.FORWARD, 'backbone': Backbone.MLP, 'estimator': Estimator.ANCHOR, 'freeze_estimator': True, 'anchor_outlier_threshold': anchor_trans_thresh, 'epochs': 0, 'lr': 0.01, 'backbone_pretrain_epochs': 150, 'label_smoothing': 0.0, 'batch_size': 32, 'device': 'cpu', 'id': '20201118_164657', 'log_step': 32, 'results_dir': '/Users/nick/uni/comp5328/assignment2/comp-5328-assignment-two/code/results', 'wandb': False}
config = argparse.Namespace()
vars(config).update(config_dict)

generate_trans_cifar(config)



backbone:
MLPBackbone(
  (fc_0): Linear(in_features=3072, out_features=1537, bias=True)
  (act_0): ReLU()
  (dropout_0): Dropout(p=0.5, inplace=False)
  (fc_1): Linear(in_features=1537, out_features=1537, bias=True)
  (act_1): ReLU()
  (fc_2): Linear(in_features=1537, out_features=3, bias=True)
  (sm): Softmax(dim=-1)
)
pretraining backbone:
pretrain/epoch: 0.3307 pretrain/loss: 1.1017
pretrain/epoch: 0.6720 pretrain/loss: 1.0939
pretrain/epoch: 0.9920 pretrain/loss: 1.1065
pretrain/epoch: 1.3307 pretrain/loss: 1.1111
pretrain/epoch: 1.6720 pretrain/loss: 1.0958
pretrain/epoch: 1.9920 pretrain/loss: 1.0994
pretrain/epoch: 2.3307 pretrain/loss: 1.0983
pretrain/epoch: 2.6720 pretrain/loss: 1.1027
pretrain/epoch: 2.9920 pretrain/loss: 1.1050
pretrain/epoch: 3.3307 pretrain/loss: 1.0993
pretrain/epoch: 3.6720 pretrain/loss: 1.0951
pretrain/epoch: 3.9920 pretrain/loss: 1.0992
pretrain/epoch: 4.3307 pretrain/loss: 1.0875
pretrain/epoch: 4.6720 pretrain/loss: 1.0943
pretrain/epoch: 4.9920 pre

pretrain/epoch: 41.3307 pretrain/loss: 1.0618
pretrain/epoch: 41.6720 pretrain/loss: 1.0369
pretrain/epoch: 41.9920 pretrain/loss: 1.0393
pretrain/epoch: 42.3307 pretrain/loss: 1.1019
pretrain/epoch: 42.6720 pretrain/loss: 1.0979
pretrain/epoch: 42.9920 pretrain/loss: 1.0874
pretrain/epoch: 43.3307 pretrain/loss: 1.0720
pretrain/epoch: 43.6720 pretrain/loss: 1.0917
pretrain/epoch: 43.9920 pretrain/loss: 1.0725
pretrain/epoch: 44.3307 pretrain/loss: 1.0913
pretrain/epoch: 44.6720 pretrain/loss: 1.0786
pretrain/epoch: 44.9920 pretrain/loss: 1.0913
pretrain/epoch: 45.3307 pretrain/loss: 1.0779
pretrain/epoch: 45.6720 pretrain/loss: 1.0799
pretrain/epoch: 45.9920 pretrain/loss: 1.0672
pretrain/epoch: 46.3307 pretrain/loss: 1.0537
pretrain/epoch: 46.6720 pretrain/loss: 1.0736
pretrain/epoch: 46.9920 pretrain/loss: 1.0543
pretrain/epoch: 47.3307 pretrain/loss: 1.0803
pretrain/epoch: 47.6720 pretrain/loss: 1.0796
pretrain/epoch: 47.9920 pretrain/loss: 1.0548
pretrain/epoch: 48.3307 pretrain/l

KeyboardInterrupt: 